In [1]:
from astropy.table import Table
from astropy.io import fits
from scipy.interpolate import interp1d,interp2d
from yattag import Doc
import re
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sea
import numpy as np
import pandas as pd
from shutil import copyfile
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({'xtick.direct'
               'ion': 'in','xtick.top':True,'xtick.minor.visible': True,
               'ytick.direction': "in",'ytick.right': True,'ytick.minor.visible': True})
cmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, 
                             light=1.2, as_cmap=True)

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
oned = glob('/Volumes/Vince_homedrive/Extractions/COMBINED/1D/PNG/*')
twod = glob('/Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/*')

In [6]:
cleardir = glob('/Volumes/Vince_homedrive/Extractions/clear*')

In [64]:
fields = np.array([])

for i in range(len(cleardir)):
    files = glob(cleardir[i] + '/1D/PNG/*')
    last = [re.split('_',U)[1] for U in files]
    first = [re.split('-',U)[1] for U in last]
    fid = [re.split('/',U)[-1] for U in first]
    fields = np.append(fields,fid)

In [66]:
southfields = ['ERSPRIME','GS1','GS2','GS3','GS4','GS5']
northfields = ['GN1','GN2','GN3','GN4','GN5','GN7']

{'ERSPRIME',
 'GN1',
 'GN2',
 'GN3',
 'GN4',
 'GN5',
 'GN7',
 'GS1',
 'GS2',
 'GS3',
 'GS4',
 'GS5'}

In [179]:
IDS = np.array([])
IDN = np.array([])

for i in range(len(cleardir)):
    files = glob(cleardir[i] + '/1D/FITS/*')
    last = [re.split('_',U)[2] for U in files]
    nums1 = [U[:-8] for U in last]
    nums1 = [int(U) for U in nums1]

    last = [re.split('_',U)[1] for U in files]
    first = [re.split('-',U)[1] for U in last]
    fid1 = [re.split('/',U)[-1] for U in first]
    
    files = glob(cleardir[i] + '/2D/PNG/*')
    img = [U.strip(cleardir[i] + '/2D/')[4:] for U in files]
    last = [re.split('_',U)[2] for U in files]
    nums2 = [U[:-7] for U in last]
    nums2 = [int(U) for U in nums2]
    
    last = [re.split('_',U)[1] for U in files]
    first = [re.split('-',U)[1] for U in last]
    fid2 = [re.split('/',U)[-1] for U in first]

    south = []
    north = []
    
    for ii in range(len(fid1)):
        if fid1[ii][1] == 'N':
            north.append(nums1[ii])
        else:
            south.append(nums1[ii])

    for ii in range(len(fid2)):
        if fid2[ii][1] == 'N':
            north.append(nums2[ii])
        else:
            south.append(nums2[ii])
            
    IDS = np.append(IDS,south)
    IDN = np.append(IDN,north)

In [180]:
IDS=IDS.astype(int)
IDN=IDN.astype(int)

In [181]:
IDXs = np.sort(list(set(IDS)))
IDXn = np.sort(list(set(IDN)))

In [182]:
IDXs.size , IDXn.size

(2223, 2572)

In [186]:
SDF = pd.DataFrame({'id' : IDXs,'2D': np.repeat('',IDXs.size),'1D': np.repeat('',IDXs.size)})
NDF = pd.DataFrame({'id' : IDXn,'2D': np.repeat('',IDXn.size),'1D': np.repeat('',IDXn.size)})

In [214]:
oneDs = np.array(np.repeat('',IDXs.size), dtype=object)
oneDn = np.array(np.repeat('',IDXn.size), dtype=object)

twoDs = np.array(np.repeat('',IDXs.size), dtype=object)
twoDn = np.array(np.repeat('',IDXn.size), dtype=object)

twoDalls = np.array(np.repeat('',IDXs.size), dtype=object)
twoDalln = np.array(np.repeat('',IDXn.size), dtype=object)

for i in range(len(cleardir)):
    files = glob(cleardir[i] + '/2D/PNG/*')
    last = [re.split('_',U)[2] for U in files]
    nums = [U[:-7] for U in last]
    nums = [int(U) for U in nums]
    
    last = [re.split('_',U)[1] for U in files]
    first = [re.split('-',U)[1] for U in last]
    fid = [re.split('/',U)[-1] for U in first]
    
    for ii in range(len(fid)):
        if fid[ii][1] == 'N':
            idx = NDF.index[NDF.id == nums[ii]].values[0]
            twoDn[idx] = twoDn[idx] + files[ii] +','
        else:
            idx = SDF.index[SDF.id == nums[ii]].values[0]
            twoDs[idx] = twoDs[idx] + files[ii] +','
            
    files = glob(cleardir[i] + '/1D/FITS/*')
    last = [re.split('_',U)[2] for U in files]
    nums = [U[:-8] for U in last]
    nums = [int(U) for U in nums]

    last = [re.split('_',U)[1] for U in files]
    first = [re.split('-',U)[1] for U in last]
    fid = [re.split('/',U)[-1] for U in first]
    
    for ii in range(len(fid)):
        if fid[ii][1] == 'N':
            idx = NDF.index[NDF.id == nums[ii]].values[0]
            oneDn[idx] = oneDn[idx] + files[ii] +','
        else:
            idx = SDF.index[SDF.id == nums[ii]].values[0]
            oneDs[idx] = oneDs[idx] + files[ii] +','
            
for i in range(len(twod)):
    nums = int(twod[i].split('_')[2])
    
    fid = twod[i].split('_')[1].split('/')[-1].split('-')[0]
    
    if fid[1] == 'N':        
        if nums in NDF.id.values:
            idx = NDF.index[NDF.id == nums].values[0]
            twoDalln[idx] = twoDalln[idx] + twod[i] +','
    else:
        if nums in SDF.id.values:
            idx = SDF.index[SDF.id == nums].values[0]
            twoDalls[idx] = twoDalls[idx] + twod[i] +','
            

In [216]:
SDF['2D_all']=twoDalls
NDF['2D_all']=twoDalln

SDF['2D']=twoDs
NDF['2D']=twoDn

SDF['1D']=oneDs
NDF['1D']=oneDn

In [219]:
SDF.to_pickle('south_DF.pkl')
NDF.to_pickle('north_DF.pkl')

# image dataframe

In [ ]:
SDF = pd.read_pickle('south_DF.pkl')
NDF = pd.read_pickle('north_DF.pkl')

In [251]:
sdat=np.zeros(len(SDF['id'].values))
ndat=np.zeros(len(NDF['id'].values))
SIDF = pd.DataFrame({'id':SDF['id'].values,'stack': sdat,'ind_spec': sdat,
                     '2d_all': sdat,'2d_ind': sdat,'z_phot': sdat,'field': sdat})

NIDF = pd.DataFrame({'id':NDF['id'].values,'stack': ndat,'ind_spec': ndat,
                     '2d_all': ndat,'2d_ind': ndat,'z_phot': ndat,'field': ndat})

In [3]:
SIDF = pd.read_pickle('south_img_DF.pkl')
NIDF = pd.read_pickle('north_img_DF.pkl')

# Copy files over

In [230]:
for i in range(len(twod)):
    fid = twod[i].split('/')[-1].split('-')[0]
    fname = twod[i].split('/')[-1]
    
    if fid[1] == 'N':
        dst = '/Volumes/Vince_homedrive/Extractions/Plots/North/2D/stack/%s' % fname

    else:
        dst = '/Volumes/Vince_homedrive/Extractions/Plots/South/2D/stack/%s' % fname

    copyfile(twod[i], dst)

In [227]:
twod[0].split('/')[-1]

'ERSPRIME-G102_36794_stack.png'

In [231]:
twoDs = np.array(np.repeat('',IDXs.size), dtype=object)
twoDn = np.array(np.repeat('',IDXn.size), dtype=object)

for i in range(len(cleardir)):
    files = glob(cleardir[i] + '/2D/PNG/*')
    last = [re.split('_',U)[2] for U in files]
    nums = [U[:-7] for U in last]
    nums = [int(U) for U in nums]
    
    last = [re.split('_',U)[1] for U in files]
    first = [re.split('-',U)[1] for U in last]
    fid = [re.split('/',U)[-1] for U in first]
    
    for ii in range(len(fid)):
        fname = files[ii].split('/')[-1]

        if fid[ii][1] == 'N':
            dst = '/Volumes/Vince_homedrive/Extractions/Plots/North/2D/spec/%s' % fname
        else:
            dst = '/Volumes/Vince_homedrive/Extractions/Plots/South/2D/spec/%s' % fname

            
        copyfile(files[ii], dst)

# Enter values in dataframe

In [2]:
goodss = Table.read('/Users/Vince.ec/catalogs_for_CLEAR/goodss_3dhst.v4.1.cats/Eazy/goodss_3dhst.v4.1.zout',
                   format = 'ascii').to_pandas()
goodsn = Table.read('/Users/Vince.ec/catalogs_for_CLEAR/goodsn_3dhst.v4.1.cats/Eazy/goodsn_3dhst.v4.1.zout',
                   format = 'ascii').to_pandas()

In [252]:
sz = sdat
nz = ndat

for i in range(len(SIDF.index)):
    sz[i] = goodss[goodss.id == SIDF['id'][i]]['z_m1'].values[0]
    
for i in range(len(NIDF.index)):
    nz[i] = goodsn[goodsn.id == NIDF['id'][i]]['z_m1'].values[0]

In [255]:
NIDF['z_phot'] = nz
SIDF['z_phot'] = sz

In [282]:
spth2a = '/Volumes/Vince_homedrive/Extractions/Plots/South/2D/stack/'
spth2i = '/Volumes/Vince_homedrive/Extractions/Plots/South/2D/spec/'
spth1a = '/Volumes/Vince_homedrive/Extractions/Plots/South/1D/stack/'
spth1i = '/Volumes/Vince_homedrive/Extractions/Plots/South/1D/spec/'

npth2a = '/Volumes/Vince_homedrive/Extractions/Plots/North/2D/stack/'
npth2i = '/Volumes/Vince_homedrive/Extractions/Plots/North/2D/spec/'
npth1a = '/Volumes/Vince_homedrive/Extractions/Plots/North/1D/stack/'
npth1i = '/Volumes/Vince_homedrive/Extractions/Plots/North/1D/spec/'

for i in range(len(SIDF.index)):
# for i in range(10):
    if len(glob(spth2a + '*' + str(SIDF['id'][i]) + '*')) > 0:
        twodnames = glob(spth2a + '*' + str(SIDF['id'][i]) + '*')
        SIDF['2d_all'][i] = twodnames
        fids = []
        
        for ii in range(len(twodnames)):
            fids.append(twodnames[ii].split('/')[-1].split('-')[0])
    
        SIDF['field'][i] = fids
    
    else:
        SIDF['2d_all'][i] = 'none'
        SIDF['field'][i] = 'none'
        
    if len(glob(spth2i + '*' + str(SIDF['id'][i]) + '*')) > 0:
        SIDF['2d_ind'][i] = glob(spth2i + '*' + str(SIDF['id'][i]) + '*')

    else:
        SIDF['2d_ind'][i] = 'none'
        
    if len(glob(spth1a + '*' + str(SIDF['id'][i]) + '*')) > 0:
        SIDF['stack'][i] = glob(spth1a + '*' + str(SIDF['id'][i]) + '*')
    
    else:
        SIDF['stack'][i] = 'none'
    
    if len(glob(spth1i + '*' + str(SIDF['id'][i]) + '*')) > 0:
        SIDF['ind_spec'][i] = glob(spth1i + '*' + str(SIDF['id'][i]) + '*')
    
    else:
        SIDF['ind_spec'][i] = 'none'
  
###################################

for i in range(len(NIDF.index)):
    if len(glob(npth2a + '*' + str(NIDF['id'][i]) + '*')) > 0:
        twodnames = glob(npth2a + '*' + str(NIDF['id'][i]) + '*')
        NIDF['2d_all'][i] = twodnames
        fids = []
        
        for ii in range(len(twodnames)):
            fids.append(twodnames[ii].split('/')[-1].split('-')[0])
    
        NIDF['field'][i] = fids
    
    else:
        NIDF['2d_all'][i] = 'none'
        NIDF['field'][i] = 'none'
        
    if len(glob(npth2i + '*' + str(NIDF['id'][i]) + '*')) > 0:
        NIDF['2d_ind'][i] = glob(npth2i + '*' + str(NIDF['id'][i]) + '*')

    else:
        NIDF['2d_ind'][i] = 'none'
        
    if len(glob(npth1a + '*' + str(NIDF['id'][i]) + '*')) > 0:
        NIDF['stack'][i] = glob(npth1a + '*' + str(NIDF['id'][i]) + '*')
    
    else:
        NIDF['stack'][i] = 'none'
    
    if len(glob(npth1i + '*' + str(NIDF['id'][i]) + '*')) > 0:
        NIDF['ind_spec'][i] = glob(npth1i + '*' + str(NIDF['id'][i]) + '*')
    
    else:
        NIDF['ind_spec'][i] = 'none'

/Users/Vince.ec/anaconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Vince.ec/anaconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Vince.ec/anaconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Vince.ec/anaconda2/envs/astroconda/lib/python2.7/site-packages/

In [284]:
SIDF.to_pickle('south_img_DF.pkl')
NIDF.to_pickle('north_img_DF.pkl')